In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [2]:
import numpy
import keras
from keras.utils.np_utils import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda

from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *

In [3]:
from Hypers import *
from Utils import *
from Preprocessing import *
from Generator import *
from Models import *

In [4]:
data_root_path = None
embedding_path = None

In [5]:
news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict = read_news(data_root_path)
news_title,news_vert,news_subvert,news_entity,news_content=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict)

title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)
content_word_embedding_matrix, have_word = load_matrix(embedding_path,content_dict)

In [6]:
train_session = read_train_clickhistory(news_index,data_root_path,'MINDlarge_train/behaviors.tsv')
train_user = parse_user(news_index,train_session)
train_sess, train_user_id, train_label = get_train_input(news_index,train_session)

news_fetcher = NewsFetcher(news_title,news_content,news_vert,news_subvert,news_entity)

In [7]:
test_session = read_test_clickhistory_noclk(news_index,data_root_path,'MINDlarge_test/behaviors.tsv')
test_user = parse_user(news_index,test_session)
test_impressions, test_userids = get_test_input(news_index,test_session)

In [8]:
train_generator = get_hir_train_generator(news_fetcher,train_user['click'],train_user_id,train_sess,train_label,32)
news_generator = get_hir_news_generator(news_fetcher,32)

In [9]:
model,news_encoder,user_encoder, = create_model(title_word_embedding_matrix,content_word_embedding_matrix,entity_dict,category_dict,subcategory_dict)
model.fit_generator(train_generator,epochs=3,verbose=False)

In [10]:
news_scoring = news_encoder.predict_generator(news_generator,verbose=1)
test_user_generator = get_hir_user_generator(news_fetcher,test_user['click'],32)
test_user_scoring = user_encoder.predict_generator(test_user_generator,verbose=1)
dump_result(test_impressions,news_scoring,test_user_scoring)

74086/74086 [==============================] - 2801s 38ms/step
